# Protocol timer

## Initialization

### Load Libraries

In [1]:
from IPython.core.getipython import get_ipython
import pandas as pd

### Define functions

In [2]:
def create_new_cell(contents):
    shell = get_ipython()

    payload = dict(
        source='set_next_input',
        text=contents,
        replace=False,
    )
    shell.payload_manager.write_payload(payload, single=False)

In [3]:
new_cells = ["print('hello')", "print('world')"]

## Results

In [4]:
for content in new_cells:
  create_new_cell(content)


#-----------------------------------------------------
# Sample printouts will be below
#-----------------------------------------------------

In [ ]:
print('world')

In [ ]:
print('hello')